In [1]:
import cv2
import Detect_inator
from detect import *
import Model_structure
import model

2024-11-09 13:34:22.673811: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-09 13:34:22.688202: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731152062.705427   86314 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731152062.711319   86314 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-09 13:34:22.728828: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
model_my = model.load_model('dataset/yolov3.cfg', 'dataset/yolov3.weights')

Skipping Sequential(
  (conv_81): Conv2d(1024, 18, kernel_size=(1, 1), stride=(1, 1))
) layer
Skipping Sequential(
  (conv_93): Conv2d(512, 18, kernel_size=(1, 1), stride=(1, 1))
) layer
Skipping Sequential(
  (conv_105): Conv2d(256, 18, kernel_size=(1, 1), stride=(1, 1))
) layer


In [3]:
img = cv2.imread('dataset/test/volley-54-_png.rf.98b1cfe6e580683422b2ad9f0bc9eae7.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [4]:
boxes = detect_image(model_my, img)
print(boxes)

[]


In [ ]:
hyper, model = Model_structure.create_modules()

In [ ]:
model[105][0].bias.numel()

255

In [ ]:
len(boxes)

12

In [ ]:
options = dict()
options['gpus'] = '0,1,2,3'
options['num_workers'] = '10'
path = 'dataset/yolov3.cfg'
options.pop(0)

KeyError: 0

In [ ]:
smth = {'gpus': '0,1,2,3', 'num_workers': '10'}